In [1]:
import org.apache.spark.SparkConf

In [2]:
val conf = new SparkConf()

conf = org.apache.spark.SparkConf@cc5b3b6


org.apache.spark.SparkConf@cc5b3b6

In [3]:
import org.apache.spark.sql.SparkSession

In [4]:
val spark = SparkSession.builder
    .appName("Tatiana Gavrikova")
    .config(conf)
    .getOrCreate()

spark = org.apache.spark.sql.SparkSession@69e017fa


org.apache.spark.sql.SparkSession@69e017fa

In [7]:
import sys.process._

In [8]:
"hdfs dfs -ls /labs/slaba05/" !

Found 2 items
-rw-r--r--   3 hdfs hdfs   24271415 2022-01-06 18:46 /labs/slaba05/lab05_test.csv
-rw-r--r--   3 hdfs hdfs  176160768 2022-01-06 18:46 /labs/slaba05/lab05_train.csv


0

In [9]:
"hdfs dfs -head /labs/slaba05/lab05_train.csv" !

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_

0

In [12]:
val df_train = spark.read.format("csv")
    .option("dilimeter", ",")
    .option("inferSchema", "true")
    .option("header", "true")
    .load("/labs/slaba05/lab05_train.csv")

df_train = [_c0: int, ID: int ... 115 more fields]


lastException: Throwable = null


[_c0: int, ID: int ... 115 more fields]

In [13]:
val df_test = spark.read
    .format("csv")
    .option("inferSchema", "true")
    .option("dilimeter", ",")
    .option("header", "true")
    .load("/labs/slaba05/lab05_test.csv")

df_test = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

# Подготовка данных

## AGE (months) -> AGE_GROUP

Преобразую возраст в группы как в 4 лабе

In [15]:
import org.apache.spark.sql.functions.udf

In [16]:
import org.apache.spark.sql.functions.col

In [17]:
val ageToGroup = udf((age: Int) => if (age < 288) 0 else if (age < 408) 1 else if (age < 528) 2 else if (age < 648) 3 else 4)

ageToGroup = UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))

In [20]:
val df_train_1 = df_train.withColumn("AGE_GROUP", ageToGroup(col("AGE")))

df_train_1 = [_c0: int, ID: int ... 116 more fields]


[_c0: int, ID: int ... 116 more fields]

In [21]:
val df_test_1 = df_test.withColumn("AGE_GROUP", ageToGroup(col("AGE")))

df_test_1 = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

# Готовим пайплайн

* OneHotEncoder 
* VectorAssembler
* GBTClassifier

In [22]:
import org.apache.spark.ml.feature.OneHotEncoder

In [23]:
val ohEncoder = new OneHotEncoder().setDropLast(true).setInputCol("AGE_GROUP").setOutputCol("AGE_VEC")

ohEncoder = oneHot_b31c9ac94d49


oneHot_b31c9ac94d49

In [24]:
import org.apache.spark.ml.feature.VectorAssembler

In [25]:
val assembler = new VectorAssembler()
    .setInputCols(Array(
        "CR_PROD_CNT_IL",
        "TURNOVER_DYNAMIC_IL_1M",
        "REST_DYNAMIC_FDEP_1M",
        "REST_DYNAMIC_SAVE_3M",
        "CR_PROD_CNT_VCU",
        "REST_AVG_CUR",
        "CR_PROD_CNT_TOVR",
        "CR_PROD_CNT_PIL",
        "TURNOVER_CC",
        "TURNOVER_PAYM",
        "CR_PROD_CNT_CC",
        "REST_DYNAMIC_FDEP_3M",
        "REST_DYNAMIC_IL_1M",
        "CR_PROD_CNT_CCFP",
        "REST_DYNAMIC_CUR_1M",
        "REST_AVG_PAYM",
        "LDEAL_GRACE_DAYS_PCT_MED",
        "REST_DYNAMIC_CUR_3M",
        "TURNOVER_DYNAMIC_CUR_1M",
        "REST_DYNAMIC_PAYM_3M",
        "REST_DYNAMIC_IL_3M",
        "TURNOVER_DYNAMIC_IL_3M",
        "REST_DYNAMIC_PAYM_1M",
        "TURNOVER_DYNAMIC_CUR_3M",
        "CLNT_SETUP_TENOR",
        "TURNOVER_DYNAMIC_PAYM_3M",
        "TURNOVER_DYNAMIC_PAYM_1M",
        "REST_DYNAMIC_CC_1M",
        "TURNOVER_DYNAMIC_CC_1M",
        "REST_DYNAMIC_CC_3M",
        "TURNOVER_DYNAMIC_CC_3M",
        "AGE_VEC"
    )).setOutputCol("features")

assembler = vecAssembler_1994daa76a37


vecAssembler_1994daa76a37

In [26]:
import org.apache.spark.ml.classification.GBTClassifier

In [27]:
val gbt = new GBTClassifier().setFeaturesCol("features").setLabelCol("TARGET")

gbt = gbtc_ea3e56299241


gbtc_ea3e56299241

In [28]:
import org.apache.spark.ml.Pipeline

In [30]:
val pipeline = new Pipeline().setStages(Array(ohEncoder, assembler, gbt))

pipeline = pipeline_6131e386cc36


pipeline_6131e386cc36

# Обучаем

In [31]:
val model = pipeline.fit(df_train_1)

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 1 in stage 4.0 failed 4 times, most recent failure: Lost task 1.3 in stage 4.0 (TID 11, spark-node-2.newprolab.com, executor 1): java.lang.NullPointerException: Value at index 0 is null
	at org.apache.spark.sql.Row$class.getAnyValAs(Row.scala:474)
	at org.apache.spark.sql.Row$class.getDouble(Row.scala:250)
	at org.apache.spark.sql.catalyst.expressions.GenericRow.getDouble(rows.scala:166)
	at org.apache.spark.ml.feature.OneHotEncoderCommon$$anonfun$11$$anonfun$apply$1.apply$mcDI$sp(OneHotEncoderEstimator.scala:486)
	at org.apache.spark.ml.feature.OneHotEncoderCommon$$anonfun$11$$anonfun$apply$1.apply(OneHotEncoderEstimator.scala:486)
	at org.apache.spark.ml.feature.OneHotEncoderCommon$$anonfun$11$$anonfun$apply$1.apply(OneHotEncoderEstimator.scala:486)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(Traversab

Надо почистить данные 

In [32]:
df_train_1.summary().show(100, 100, true)

-RECORD 0-------------------------------------------------------
 summary                     | count                            
 _c0                         | 320764                           
 ID                          | 320764                           
 CR_PROD_CNT_IL              | 320764                           
 AMOUNT_RUB_CLO_PRC          | 286214                           
 PRC_ACCEPTS_A_EMAIL_LINK    | 139950                           
 APP_REGISTR_RGN_CODE        | 54850                            
 PRC_ACCEPTS_A_POS           | 139950                           
 PRC_ACCEPTS_A_TK            | 139950                           
 TURNOVER_DYNAMIC_IL_1M      | 320763                           
 CNT_TRAN_AUT_TENDENCY1M     | 69535                            
 SUM_TRAN_AUT_TENDENCY1M     | 69535                            
 AMOUNT_RUB_SUP_PRC          | 286213                           
 PRC_ACCEPTS_A_AMOBILE       | 139950                           
 SUM_TRAN_AUT_TENDENCY3M 

lastException: Throwable = null


In [39]:
val df_train_2 = df_train_1.filter(col("TURNOVER_CC").isNotNull)

df_train_2 = [_c0: int, ID: int ... 116 more fields]


lastException: Throwable = null


[_c0: int, ID: int ... 116 more fields]

In [40]:
val model = pipeline.fit(df_train_2)

model = pipeline_6131e386cc36


pipeline_6131e386cc36

# Делаем предсказания

In [41]:
val predictions = model.transform(df_test_1)

predictions = [_c0: int, ID: int ... 120 more fields]


[_c0: int, ID: int ... 120 more fields]

In [42]:
predictions.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [44]:
predictions.select(col("probability"), col("prediction")).take(1)

Array([[0.8558955133349228,0.14410448666507725],0.0])

In [47]:
import org.apache.spark.ml.linalg.Vector

In [48]:
val getProb = udf((prob: Vector) => prob(1))

getProb = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [49]:
val res = predictions
    .withColumn("prob", getProb(col("probability")))
    .select(col("ID").as("id"), col("prob").as("target"))
    .repartition(1)

res = [id: int, target: double]


[id: int, target: double]

In [50]:
res.write
    .option("header", true)
    .option("delimiter","\t")
    .csv("lab05.csv")

In [51]:
"hdfs dfs -ls lab05.csv" !

Found 2 items
-rw-r--r--   3 tatiana.gavrikova tatiana.gavrikova          0 2022-11-04 23:40 lab05.csv/_SUCCESS
-rw-r--r--   3 tatiana.gavrikova tatiana.gavrikova    1195350 2022-11-04 23:40 lab05.csv/part-00000-60fc8fc4-04e5-49f0-9f65-522dd85233c6-c000.csv


0

In [52]:
"hdfs dfs -copyToLocal lab05.csv/part-00000-60fc8fc4-04e5-49f0-9f65-522dd85233c6-c000.csv lab05.csv" !

0

In [53]:
"ls -l" !

total 69880
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova     7500 Oct 14 16:43 lab01.ipynb
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova       84 Oct 14 16:43 lab01.json
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova    17874 Oct 24 17:55 lab02.ipynb
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova      462 Oct 23 15:44 lab02.json
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova 70176888 Oct 29 02:51 lab03.csv
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova    13872 Oct 29 03:00 lab03.ipynb
-rw-r--r-- 1 tatiana.gavrikova tatiana.gavrikova  1195350 Nov  4 23:40 lab05.csv
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova   114583 Nov  4 23:40 lab05.ipynb
-rw-rw-r-- 1 tatiana.gavrikova tatiana.gavrikova     5127 Oct 14 16:28 SparkSession.ipynb


0

# Не забываем погасить контекст

In [54]:
spark.stop()